In [ ]:
import numpy as np
import pandas as pd

from lazypredict.Supervised import LazyClassifier
from sklearn.feature_selection import VarianceThreshold

In [ ]:
folder = "../data/"

videos = [
    "Koi_5652_952_540",
    "Pigeons_29033_960_540_300f",
    "Pigeons_4927_960_540_600f",
    "Pigeons_8234_1280_720",
    "Pigs_49651_960_540_500f",
]

features = [
    "AE",
    "HOG",
    "LBP",
    "MN2",
    "RGB",
]

folds = ["h1", "h2"]

In [ ]:
results = None

for v in videos:
    for f in features:
        print(v, f)
        
        h1 = pd.read_csv(f"{folder}h1_{v}_{f}.csv")
        h2 = pd.read_csv(f"{folder}h2_{v}_{f}.csv")
        
        h1.drop(["Frames"], axis=1, inplace=True, errors='ignore')
        h2.drop(["Frames"], axis=1, inplace=True, errors='ignore')
        
        accuracies = None

        for (train, test) in ((h1, h2), (h2, h1)):
            X_train = train.iloc[: , :-1]
            y_train = train.iloc[: , -1]
                    
            X_test = test.iloc[: , :-1]
            y_test = test.iloc[: , -1]
            
            selector = VarianceThreshold()
            X_train = selector.fit_transform(X_train)
            X_test = selector.transform(X_test)
            
            clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
            models, predictions = clf.fit(X_train, X_test, y_train, y_test)
            models.sort_index(inplace=True)
            
            if accuracies is None:
                accuracies = models.iloc[ : , :1]
            else:
                accuracies +=  models.iloc[ : , :1]
                
        accuracies /= len(folds)
            
        if results is None:
            results = accuracies
            results.columns = [f"{v}_{f}"]
        else:
            results[f"{v}_{f}"] = accuracies
            
    display(results)    

results = results.dropna()
display(results)

results.to_csv("animal_classification_results.csv")